Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.

In [1]:
#импортируем необходимую библиотеку
import pandahouse as ph

In [2]:
# создаем соединение с базой данных ClickHouse
connection1 = {
                    'host': 'https://clickhouse.lab.karpov.courses',
                    'database': 'default',
                    'user': 'student', 
                    'password': 'dpo_python_2020'
               }

In [3]:
# запрос SELECT
query1 = """
    SELECT uniqExact(st_id) AS count_students
FROM (
    SELECT
        st_id,
        SUM(correct) OVER(PARTITION BY st_id) AS sum_correct
    FROM (
        SELECT *
        FROM default.peas
        WHERE 
            toMonth(timest) = (SELECT toMonth(MAX(timest)) FROM default.peas) 
            AND toYear(timest) = (SELECT toYear(MAX(timest)) FROM default.peas)
        )
    )
WHERE sum_correct >= 20
"""

In [4]:
df1 = ph.read_clickhouse(query1, connection=connection1)

# выводим результат
print(f"Количество усердных студентов: {df1['count_students'][0]}")

Количество усердных студентов: 136


Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

Необходимо в _одном запросе_ выгрузить следующую информацию о группах пользователей:

+ ARPU 
+ ARPAU 
+ CR в покупку 
+ СR активного пользователя в покупку 
+ CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике

ARPU считается относительно всех пользователей, попавших в группы.
**Активным** считается пользователь, за все время решивший **больше 10 задач правильно** в любых дисциплинах.
**Активным по математике** считается пользователь, за все время решивший **2 или больше задач** правильно по математике.

In [5]:
query2 = """
WITH active AS (
    SELECT
    st_id,
    IF(count_correct > 10, 1, 0) AS active,
    IF(count_math_correct >= 2, 1, 0) AS active_math
FROM (
    SELECT 
        DISTINCT st_id,
        SUM(correct) OVER (PARTITION BY st_id) AS count_correct,
        sumIf(correct, subject = 'Math') OVER (PARTITION BY st_id) AS count_math_correct 
    FROM default.peas
    )
),
money AS (
SELECT 
    st_id,
    sum_money,
    sum_math_money,
    IF(sum_money > 0, 1, 0) AS paid,
    IF(sum_math_money > 0, 1, 0) AS paid_math
FROM (
    SELECT 
        DISTINCT st_id,
        SUM(money) OVER (PARTITION BY st_id) AS sum_money,
        sumIf(money, subject = 'Math') OVER (PARTITION BY st_id) AS sum_math_money
    FROM default.final_project_check
)
)

SELECT
    test_grp,
    ROUND((SUM(C.sum_money) / uniqExact(A.st_id)), 2) AS ARPU,
    sumIf(C.sum_money, B.active = 1) / SUM(B.active) AS ARPAU,
    ROUND(SUM(C.paid) / uniqExact(A.st_id) * 100, 2)  AS CR,
    ROUND(sumIf(C.paid, B.active = 1) / SUM(B.active) * 100, 2) AS CR_active,
    ROUND(sumIf(C.paid_math, B.active_math = 1) / SUM(B.active_math) * 100, 2) AS CR_math
FROM default.studs AS A
    LEFT JOIN active AS B ON A.st_id = B.st_id
    LEFT JOIN money AS C ON A.st_id = C.st_id
GROUP BY test_grp
"""

In [6]:
df2 = ph.read_clickhouse(query2, connection=connection1)

# выводим результат
df2

,test_grp,ARPU,ARPAU,CR,CR_active,CR_math
0,control,4540.98,10393.700787,4.92,11.02,6.12
1,pilot,11508.47,29739.583333,10.85,26.04,9.52
